In [ ]:
import matplotlib.pyplot as plt
from ipywidgets import *
import numpy as np
import pandas as pd
import seaborn as sns
import scipy

# NB: add more subjects in the data folder in neurolib package
from neurolib.utils.loadData import Dataset
from neurolib.models.aln import ALNModel

from neurolib.utils import functions as func

import brainplot as bp

plt.style.use('seaborn')
# sns.set_style("whitegrid")
sns.set_style("white")
sns.set_context("talk")

In [ ]:
ds = Dataset("gw")
# ds.Cmat = ds.Cmats[3]
# ds.Dmat = ds.Dmats[3]
model = ALNModel(Cmat=ds.Cmat, Dmat=ds.Dmat)

# add custom parameter for downsampling results
# 10 ms sampling steps for saving data, should be multiple of dt
model.params['save_dt'] = 10.0
model.params["tauA"] = 600.0
model.params["sigma_ou"] = 0.0
model.params["b"] = 20.0

model.params["Ke_gl"] = 300.0
model.params["mue_ext_mean"] = 0.2
model.params["mui_ext_mean"] = 0.1

# Sleep model from newest evolution October 2020
model.params["mue_ext_mean"] = 3.3202829454334535
model.params["mui_ext_mean"] = 3.682451894176651
model.params["b"] =  3.2021806735984186
model.params["tauA"] = 4765.3385276559875
model.params["sigma_ou"] = 0.36802952978628106
model.params["Ke_gl"] = 265.48075753153


model.params['dt'] = 0.1
model.params['duration'] = 60 * 1000 #ms
model.params["signalV"] = 80.0

model.run(bold=True)

In [ ]:
%matplotlib widget
plt.figure(figsize=(10, 4))
plt.imshow(model.output, aspect="auto", extent=[0, model.t[-1] / 1000, model.params.N, 0],
            clim=(0, 20), cmap="plasma")
cbar = plt.colorbar(extend='max', fraction=0.046, pad=0.04)
cbar.set_label("Rate $r_{exc}$ [Hz]")
plt.ylabel("Node")
plt.xlabel("Time [s]")
plt.tight_layout()
plt.show()

In [ ]:
states = bp.detectSWs(model)

In [ ]:
bp.plot_states_timeseries(model, states, cmap="plasma")

In [ ]:
involvement = bp.get_involvement(states)

In [ ]:
def plot_involvement_timeseries(model, involvement, peaks=[], local_peaks=[]):
    fig, ax = plt.subplots(1, 1, figsize=(10, 4))
    ax.set_title("Involvement of brain areas in SO events")
    ax.plot(model.t / 1000, involvement * 100, c="C0")
    ax.set_ylabel("Involvement [%]")
    ax.set_xlabel("Time [s]")
    ax.set_ylim([0, 100])
    ax.set_aspect("auto")
    if len(peaks) != 0:
        for peak in (peaks):
            ax.scatter(x=peak / 10000, y=involvement[peak] * 100, c='r')
    if len(local_peaks) != 0:
        for peak in (local_peaks):
            ax.scatter(x=peak / 10000, y=involvement[peak] * 100, c='g')
            # ax.text(x=peak / 10000, y=involvement[peak] * 100, s = f"{involvement[peak] * 100:.0f}")

In [ ]:
plot_involvement_timeseries(model, involvement)

In [ ]:
filtered_involvement = scipy.ndimage.gaussian_filter1d(involvement, 2000)

In [ ]:
plot_involvement_timeseries(model, filtered_involvement)

In [ ]:
peaks = scipy.signal.find_peaks(
        filtered_involvement, height=0.1, distance=1000)[0]

In [ ]:
plot_involvement_timeseries(model, filtered_involvement, peaks=peaks)

In [ ]:
def filter_peaks(peaks, inv, t_max, t_min=0):
    return [p for p in peaks if (inv[p] > t_min and inv[p] <= t_max)]

peaks25 = filter_peaks(peaks, involvement, 0.50, 0.0)
n_local_waves = len(peaks25)
peaks50 = filter_peaks(peaks, involvement, 0.75, 0.50)
peaks75 = filter_peaks(peaks, involvement, 1, 0.75)
n_global_waves = len(peaks50 + peaks75)

In [ ]:
plot_involvement_timeseries(model, filtered_involvement, peaks=peaks50+peaks75, local_peaks=peaks25)

In [ ]:
from neurolib.utils import atlases
from nilearn import plotting  
atlas = atlases.AutomatedAnatomicalParcellation2()
# AAL2 atlas is taken from here: https://github.com/cwatson/brainGraph
aal2_atlas = pd.read_csv("aal2_coords.csv")
coords = aal2_atlas.loc[atlas.cortex, ["x.mni", "y.mni", "z.mni"]].to_numpy()
fig, ax = plt.subplots(figsize=(10, 4))
# display = plotting.plot_glass_brain(None)
plotting.plot_markers(
    node_coords=coords, node_values=[1]*80,node_size=[150]*80, 
    node_cmap="plasma", 
    figure=fig, axes=ax)

In [ ]:
%matplotlib widget
#pip install ipympl
fig, ax = plt.subplots(2, 1, figsize=(10, 8))

ax[0].set_title("Involvement of brain areas in SO events")
ax[0].plot(model.t / 1000, filtered_involvement * 100, c="C0")
ax[0].set_ylabel("Involvement [%]")
ax[0].set_xlabel("Time [s]")
ax[0].set_ylim([0, 100])
ax[0].set_aspect("auto")

for peak in (peaks):
    ax[0].scatter(x=peak / 10000, y=filtered_involvement[peak] * 100, c='r')
line = ax[0].axvline(x=1, c='r')



def update(i=1):
    ax[1].clear()
    line.set_xdata(i/10000)
    display = plotting.plot_markers(
    node_coords=coords, node_values=states[:, i],
    node_size=[150]*80, node_cmap="plasma",
    figure=fig, axes=ax[1], colorbar=False)
    fig.canvas.draw_idle()

interact(
    update, i=SelectionSlider(
    options=peaks,
    value=peaks[0],
    disabled=False)
    );

In [ ]:
%matplotlib widget
from matplotlib import animation, rc
#pip install ipympl

rc('animation', html='html5')

fig, ax = plt.subplots(2, 1, figsize=(10, 8))

def init():
    global line
    ax[0].set_title("Involvement of brain areas in SO events")
    ax[0].plot(model.t / 1000, filtered_involvement * 100, c="C0")
    ax[0].set_ylabel("Involvement [%]")
    ax[0].set_xlabel("Time [s]")
    ax[0].set_ylim([0, 100])
    ax[0].set_aspect("auto")

    for peak in (peaks):
        ax[0].scatter(x=peak / 10000, y=filtered_involvement[peak] * 100, c='r')
    line = ax[0].axvline(x=1, c='r')

def update(i=1):
    ax[1].clear()
    line.set_xdata(peaks[i]/10000)
    display = plotting.plot_markers(
    node_coords=coords, node_values=states[:, peaks[i]],
    node_size=[150]*80, node_cmap="plasma",
    figure=fig, axes=ax[1], colorbar=False)
    fig.canvas.draw_idle()

anim = animation.FuncAnimation(fig, update, init_func=init,
                               frames=len(peaks), interval=20, blit=True)
anim.save('all_peaks_involvement_brain_plot.gif', writer='imagemagick', fps=3)

In [ ]:
%matplotlib widget
from matplotlib import animation, rc
#pip install ipympl

rc('animation', html='html5')

fig, ax = plt.subplots(2, 1, figsize=(15, 8))


inv_states = (1 - states) * -1

def init():
    global line
    ax[0].set_title("States timeseries")
    ax[0].imshow(model.output, aspect="auto", extent=[0, model.t[-1] / 1000, model.params.N, 0],
                 clim=(0, 20), cmap="plasma")
    ax[0].set_ylabel("Node")
    ax[0].set_xlabel("Time [s]")
    # ax[0].set_ylim([0, 100])
    line = ax[0].axvline(x=1, c='r')

def update(i=1):
    ax[1].clear()
    line.set_xdata(peaks[i]/10000)
    display = plotting.plot_markers(
    node_coords=coords, node_values=states[:, peaks[i]],
    node_size=[150]*80, node_cmap="plasma",
    figure=fig, axes=ax[1], colorbar=False)
    fig.canvas.draw_idle()

anim = animation.FuncAnimation(fig, update, init_func=init,
                               frames=len(peaks), interval=20, blit=True)
anim.save('all_peaks_ts_brain_plot.gif', writer='imagemagick', fps=3)

In [ ]:
from sklearn.cluster import AgglomerativeClustering

X = np.array([coords[i, :] for i in range(80) if states[i, peaks[0]] == 0])
clustering = AgglomerativeClustering(n_clusters=None, distance_threshold=50).fit(X)
clustering

clustering.labels_

In [ ]:
values, counts = np.unique(clustering.labels_, return_counts=True)
values, counts

In [ ]:
plotting.plot_markers(
    node_coords=X, node_values=clustering.labels_/values.max(),
    node_cmap="plasma")

In [ ]:
%matplotlib widget
#pip install ipympl
fig, ax = plt.subplots( 1, figsize=(10, 4))



def update(i=1):
    fig.clear()

    statesi = states[:, i]
    X = np.array([coords[n, :] for n in range(80) if statesi[n] == 0])
    clustering = AgglomerativeClustering(n_clusters=None, distance_threshold=150).fit(X)
    labels = clustering.labels_
    values, counts = np.unique(labels, return_counts=True)
    mask = labels != values[counts.argmax()]
    node_values = np.zeros_like(labels)
    node_values[mask] = 1
    # statesi[statesi == 0][labels != values[counts.argmax()]] = 0.5
    print(sum(labels == values[counts.argmax()]))
    display = plotting.plot_markers(
    node_coords=X, node_values=node_values,
    node_cmap="plasma", # node_size=[150]*80
    figure=fig, axes=ax, colorbar=False)
    fig.canvas.draw_idle()

interact(
    update, i=SelectionSlider(
    options=peaks,
    value=peaks[0],
    disabled=False)
    );